In [1]:
import numpy as np

In [2]:
class quadratic_problem:
    def __init__(self, A, b, c, H, M, q=None, r=None, tol=1e-8):
        def check_shape(x, dim=None, varname=""):
            if isinstance(x, np.ndarray):
                if dim == None or x.shape == dim:
                    return x
                else: raise TypeError(f"{varname} has shape {x.shape} expected {dim}")
            else: raise TypeError(f"{varname} is not {type({np.ndarray})}")
        
        self.tol = tol
            
        # shape of A is assumed to be correct
        self.A = check_shape(A, varname="A")
        m, n = A.shape
        
        self.b = check_shape(b, dim=(m,), varname="b")
        self.c = check_shape(c, dim=(n,), varname="c")
        self.H = check_shape(H, dim=(n,n), varname="H")
        self.M = check_shape(M, dim=(m,m), varname="M")
        
        self.q = check_shape(q, dim=(n,), varname="q") if q else np.zeros(n)
        self.r = check_shape(r, dim=(n,), varname="r") if q else np.zeros(n)
        
        # init solutions
        self.x = np.zeros(n)
        self.y = np.zeros(m)
        self.z = np.zeros(n)
        
        # init deltas
        self.dx = np.zeros(n)
        self.dy = np.zeros(m)
        self.dz = np.zeros(n)
        
        # init B, N (temporary)
        self.B = np.full(n, True)
        self.N = np.full(n, False)

In [3]:
A1 = np.random.rand(100,200)
b1 = np.random.rand(100)
c1 = np.random.rand(200)
H1 = np.random.rand(200,200)
M1 = np.random.rand(100,100)

In [4]:
Q = quadratic_problem(A1, b1, c1, H1, M1)

## Primal Active Set

In [5]:
def primal_active_set_base(self, l):
    # compute dx, dy, dz
    # block and B_size are temporary mockups
    # all [rows,:][:,columns].transpose() should be inverted instead
    self.dx[l] = 1
    K_B = np.block([
        [self.H[self.B,:][:,self.B], self.A[:,self.B].transpose()],
        [       self.A[:,self.B]   ,         - self.M            ]
    ])
    B_size = np.sum((self.B))
    to_solve_A = K_B
    to_solve_b = - np.concatenate([self.H[self.B,:][:,l], self.A[:,l]])
    sol = np.linalg.solve(to_solve_A, to_solve_b)
    self.dx[self.B], self.dy[:] = sol[:B_size], - sol[B_size:]
    self.dz[self.N] = (
        self.H[self.N,:][:,l]*self.dx[l]
        + self.H[self.B,:][:,self.N].transpose().dot(self.dx[self.B])
        + self.A[:,self.N].transpose().dot(self.dy)
    )
    self.dz[l] = (
        self.H[l,:][:,l]*self.dx[l]
        + self.H[self.B,:][:,l].transpose().dot(self.dx[self.B])
        + self.A[:,l].transpose().dot(self.dy)
    )
    
    # compute alpha_star
    if np.allclose(dz[l], 0, rtol=tol):    # dz[l] == 0
        alpha_star = np.inf
    else:
        alpha_star = - (self.z[l] + self.r[l]) / dz[l]
    
    # compute alpha_max
    dx_i_neg = (dx < 0)
    to_min = (self.x[dx_i_neg] + self.q[dx_i_neg] / (-dx[dx_i_neg]))
    k = np.argmin(to_min)
    alpha_max = to_min[k]
    
    alpha = np.min(alpha_star, alpha_max)
    
    if np.isinf(alpha):
        # terminate primal is unbounded (dual is unfeasible)
        raise Exception("Primal is Unbounded (Dual is Unfeasible)")
    
    # compute update
    self.x[l] += alpha*self.dx[l]
    self.x[B] += alpha*self.dx[B]
    self.y[:] += alpha*self.dy
    self.z[l] += alpha*self.dz[l]
    self.z[N] += alpha*self.dz[N]
    
    if alpha == alpha_max:
        self.B[k], self.N[k] = False, True

In [6]:
def primal_active_set_intermediate(self, l):
    self.dz[l] = 1
    K_l = np.block([
        [self.H[l,:][:,l]     , self.H[self.B,:][:,l].transpose(), self.A[:,l].transpose()     ],
        [self.H[self.B,:][:,l], self.H[self.B,:][:,self.B]       , self.A[:,self.B].transpose()],
        [self.A[:,l]          , self.A[:,B]                      , - self.M                    ]
    ])
    B_size = np.sum((self.B))
    to_solve_A = K_l
    to_solve_b = np.concatenate(np.ones(1), np.zeros(B_size + self.y))
    sol = np.linalg.solve(to_solve_A, to_solve_b)
    self.dx[l], self.dx[self.B], self.dy[:] = sol[0], sol[1:B_size+1], sol[B_size+1:]
    self.dz[self.N] = (
        self.H[self.N,:][:,l]*self.dx[l]
        + self.H[self.B,:][:,self.N].transpose().dot(self.dx[self.B])
        + self.A[:,self.N].transpose().dot(self.dy)
    )
    
    # compute alpha_star
    alpha_star = - (self.z[l] + self.r[l])
    
    # compute alpha_max
    dx_i_neg = (dx < 0)
    to_min = (self.x[dx_i_neg] + self.q[dx_i_neg] / (-dx[dx_i_neg]))
    k = np.argmin(to_min)
    alpha_max = to_min[k]
    
    alpha = np.min(alpha_star, alpha_max)
    
    # compute update
    self.x[l] += alpha*self.dx[l]
    self.x[B] += alpha*self.dx[B]
    self.y[:] += alpha*self.dy
    self.z[l] += alpha*self.dz[l]
    self.z[N] += alpha*self.dz[N]
    
    if alpha == alpha_max:
        self.B[k], self.N[k] = False, True

In [7]:
def solve_primal(self):
    # find x,y,z and N,B that satisfy conditions
    while True:
        l_list = np.argwhere((self.z + self.r) < 0)
        if l_list.size == 0:
            break
        l = l_list[0]
        self.N[l] = False
        primal_active_set_base(self, l)
        while (self.z[l] + self.r[l]) < 0:
            primal_active_set_intermediate(self, l)

Non so come computare una soluzione iniziale (x,y,z) e il set B, N

In [8]:
solve_primal(Q)